In [4]:
import pandas as pd
from sqlalchemy import create_engine

In [5]:
stocks = pd.read_csv("data/all_stocks_5yr.csv")

In [6]:
stocks.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [7]:
grouped = stocks.groupby(by=['date']).mean()

grouped.head(20)

,open,high,low,close,volume
date,,,,,
2013-02-08,58.068730,58.659286,57.802352,58.368654,4.643484e+06
2013-02-11,58.355556,58.686073,57.890072,58.272196,4.092442e+06
2013-02-12,58.321741,58.771200,57.954938,58.404071,5.122822e+06
2013-02-13,58.478904,58.899515,58.040928,58.485503,5.132086e+06
2013-02-14,58.289216,58.878329,57.871977,58.473144,5.482383e+06
2013-02-15,58.485982,58.932768,57.970143,58.437170,5.734999e+06
2013-02-19,58.475575,59.076565,58.088696,58.762733,5.274365e+06
2013-02-20,58.702446,59.020518,57.765783,57.915434,5.667893e+06
2013-02-21,57.765623,58.128038,57.007764,57.505777,6.042141e+06


In [8]:
close = grouped[['close']]

close.head(20)

,close
date,
2013-02-08,58.368654
2013-02-11,58.272196
2013-02-12,58.404071
2013-02-13,58.485503
2013-02-14,58.473144
2013-02-15,58.437170
2013-02-19,58.762733
2013-02-20,57.915434
2013-02-21,57.505777


In [9]:
close.to_csv('data/cleaned_stocks.csv')

In [10]:
clean_stocks = pd.read_csv("data/cleaned_stocks.csv")
bitcoin_data = pd.read_csv("data/CBBTCUSD.csv")

In [11]:
clean_stocks.head()

,date,close
0,2013-02-08,58.368654
1,2013-02-11,58.272196
2,2013-02-12,58.404071
3,2013-02-13,58.485503
4,2013-02-14,58.473144


In [29]:
bitcoin_data = bitcoin_data.rename(columns={"DATE":"date"})

bitcoin_data.head()

,date,CBBTCUSD
0,2015-08-19,227.34
1,2015-08-20,235.56
2,2015-08-21,232.85
3,2015-08-22,231.15
4,2015-08-23,229.21


In [32]:
#Merge

merged_table = clean_stocks.merge(bitcoin_data, on='date', how='inner')

merged_table = merged_table.rename(columns={"close": 'stock_price', "CBBTCUSD": 'bitcoin_price'})

merged_table.head()

,date,stock_price,bitcoin_price
0,2015-08-19,86.399957,227.34
1,2015-08-20,84.537329,235.56
2,2015-08-21,82.040237,232.85
3,2015-08-24,78.762726,211.16
4,2015-08-25,77.847608,223.73


In [ ]:
password = ""

In [37]:
connection_string = f"postgres:{password}@localhost:5432/Bitcoin-ETL"
engine = create_engine(f'postgresql://{connection_string}')

In [38]:
#engine.execute("CREATE TABLE data(date Date, stock_price Money, bitcoin_price Money)")

ProgrammingError: (psycopg2.errors.DuplicateTable) relation "data" already exists

[SQL: CREATE TABLE data(date Date, stock_price Money, bitcoin_price Money)]
(Background on this error at: http://sqlalche.me/e/13/f405)

In [39]:
engine.table_names()

['data']

In [40]:
merged_table.to_sql(name='data', con=engine, if_exists='append', index=False)

In [44]:
test = engine.execute("SELECT data.date, data.stock_price, data.bitcoin_price FROM data LIMIT 10")

for i in test:
    print(i)

(datetime.date(2015, 8, 19), '$86.40', '$227.34')
(datetime.date(2015, 8, 20), '$84.54', '$235.56')
(datetime.date(2015, 8, 21), '$82.04', '$232.85')
(datetime.date(2015, 8, 24), '$78.76', '$211.16')
(datetime.date(2015, 8, 25), '$77.85', '$223.73')
(datetime.date(2015, 8, 26), '$80.57', '$226.81')
(datetime.date(2015, 8, 27), '$82.47', '$225.80')
(datetime.date(2015, 8, 28), '$82.52', '$231.55')
(datetime.date(2015, 8, 31), '$81.68', '$230.75')
(datetime.date(2015, 9, 1), '$79.48', '$228.22')
